## Score challenge submission

This notebook produces scores for a custom challenge set.  It scores each tasks and provides a summary score (average).

This is designed to help compare the quality of a solution against a challenge set under our control and demonstrate that the aicrowd score is consistent.

In [ ]:
import sys
import json
import re
import collections
import os
import datetime
import pandas as pd
import numpy as np

In [ ]:
challenge="ex2-from-21k-train"
submitdir="/home/jpr/projects/mpd-challenge-aicrowd/"
submissionfile="method-02-mympd-2nd-21k-2021-11-07.csv.gz"

### Load the no holdouts challenge set

In [ ]:
with open('data/{}/challenge_set_noholdout.json'.format(challenge),'r') as f:
    data = json.loads(f.read())

In [ ]:
data

In [ ]:
noholdout = pd.json_normalize(data,"playlists")

In [ ]:
noholdout

### Load challenge submission`

In [ ]:
rectracks=pd.read_csv('{}/{}'.format(submitdir, submissionfile), header=None, skiprows=1, index_col=0)


Make sure the pids match those of the challenge set.  In the case of mympd the pid range starts at 200000 and goes up in sequence of noholdout data set.

In [ ]:
rectracks.sort_values(by=0, axis='index')

## convert the noholdouts into track lists

In [ ]:
startpid=2000000

In [ ]:
[track["track_uri"] for track in noholdout.iloc[0].tracks[0:2]]

In [ ]:
len(noholdout)

R-precision is the fraction of correctly recommended tracks in the ground truth playlist as described on [the challenge site](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge#evaluation)
    

Use the [r-precesion calculation from the hello_world metrics](https://github.com/jprorama/spotify_recSys_challenge_2018/blob/f33d82715190a20fdbc998c9ff709bcabd62a55e/utils/metrics.py#L26)

In [ ]:
realtracks=pd.DataFrame()
rprec_sum = 0.0
successes = 0

for i, tracks in noholdout.head(10000).tracks.iteritems():
    pid = startpid + i;
    gtlen=len(noholdout.iloc[i].tracks)
    gttracks = [track["track_uri"] for track in noholdout.iloc[i].tracks[0:gtlen]]
    gtset = set(gttracks)
    recset = set(rectracks.iloc[i][0:gtlen])
    rprec = len(gtset & recset)/gtlen
    if rprec > 0:
        successes += 1
        #print("gt:  {}: {}".format(pid, ",".join(gttracks)))
        #print("rec: {}: {}".format(pid, ",".join(rectracks.iloc[i][0:gtlen])))
        #print("intersect: {}".format(gtset & recset))
        print("task name:     {}".format(noholdout.iloc[i]["task_name"]))
        print("intersect len: {}".format(len(gtset & recset)))
        print("playlist len:  {}".format(gtlen))
        print("playlist name: {}".format(noholdout.iloc[i]["name"]))
        print("{}: {}".format(i, rprec))
        print("tracks: {}".format(",".join([track["artist_name"] for track in noholdout.iloc[i].tracks[0:gtlen]])))
    rprec_sum = rprec_sum + rprec

#print("rprec_sum = {}".format(rprec_sum))
print("rprec = {}".format(rprec_sum/len(noholdout)))
print("succeses = {}".format(successes))
#print("{}".format(tracks[0:len(tracks)]["track_uri"]))
    